<a href="https://colab.research.google.com/github/hindia-vic/deep_learning/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install gensim
!pip install python-Levenshtein

In [19]:
# Download the .gz file from the provided URL
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

# Check if the file has been downloaded
!ls -lh reviews_Cell_Phones_and_Accessories_5.json.gz


--2024-10-16 13:38:41--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  1.67MB/s    in 25s     

2024-10-16 13:39:06 (1.76 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]

-rw-r--r-- 1 root root 44M Apr 26  2016 reviews_Cell_Phones_and_Accessories_5.json.gz


In [21]:
import gzip
import shutil

# Extract the .gz file
with gzip.open('reviews_Cell_Phones_and_Accessories_5.json.gz', 'rb') as f_in:
    with open('reviews_Cell_Phones_and_Accessories_5.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Check if the .json file has been extracted
!ls -lh reviews_Cell_Phones_and_Accessories_5.json


-rw-r--r-- 1 root root 136M Oct 16 13:40 reviews_Cell_Phones_and_Accessories_5.json


In [23]:
import json

# Open the file and read it line by line
with open('reviews_Cell_Phones_and_Accessories_5.json', 'r') as f:
    data = []
    for line in f:
        # Parse each line as a separate JSON object
        data.append(json.loads(line))

# Print a sample of the data
print(data[:2])  # Print first 2 records for preview


[{'reviewerID': 'A30TL5EWN6DFXT', 'asin': '120401325X', 'reviewerName': 'christina', 'helpful': [0, 0], 'reviewText': "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again", 'overall': 4.0, 'summary': 'Looks Good', 'unixReviewTime': 1400630400, 'reviewTime': '05 21, 2014'}, {'reviewerID': 'ASY55RVNIL0UD', 'asin': '120401325X', 'reviewerName': 'emily l.', 'helpful': [0, 0], 'reviewText': 'These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :)', 'overall': 5.0, 'summary': 'Really great product.', 'unixReviewTime': 1389657600, 'reviewTime': '01 14, 2014'}]


In [25]:
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json('reviews_Cell_Phones_and_Accessories_5.json', lines=True)

# Display the first few rows of the DataFrame
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [26]:
df.shape

(194439, 9)

In [31]:
from gensim.utils import simple_preprocess

In [32]:
text = "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again."

# Preprocess the text
print(simple_preprocess(text))

['they', 'look', 'good', 'and', 'stick', 'good', 'just', 'don', 'like', 'the', 'rounded', 'shape', 'because', 'was', 'always', 'bumping', 'it', 'and', 'siri', 'kept', 'popping', 'up', 'and', 'it', 'was', 'irritating', 'just', 'won', 'buy', 'product', 'like', 'this', 'again']


In [35]:
review_text=df.reviewText.apply(simple_preprocess)
review_text

,reviewText
0,"[they, look, good, and, stick, good, just, don..."
1,"[these, stickers, work, like, the, review, say..."
2,"[these, are, awesome, and, make, my, phone, lo..."
3,"[item, arrived, in, great, time, and, was, in,..."
4,"[awesome, stays, on, and, looks, great, can, b..."
...,...
194434,"[works, great, just, like, my, original, one, ..."
194435,"[great, product, great, packaging, high, quali..."
194436,"[this, is, great, cable, just, as, good, as, t..."
194437,"[really, like, it, becasue, it, works, well, w..."


In [40]:
from gensim.models import Word2Vec

In [41]:
#creating a model

model=Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [43]:
model.build_vocab(review_text,progress_per=1000)

In [45]:
model.train(review_text,total_examples=model.corpus_count,epochs=model.epochs)

(61507096, 83868975)

In [47]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [49]:
model.wv.most_similar("bad")

[('terrible', 0.6863459944725037),
 ('shabby', 0.6142410039901733),
 ('horrible', 0.6007099747657776),
 ('good', 0.5930659770965576),
 ('awful', 0.5505834221839905),
 ('poor', 0.537645161151886),
 ('okay', 0.5308696031570435),
 ('crappy', 0.5240293145179749),
 ('legit', 0.5179397463798523),
 ('crummy', 0.517649233341217)]

In [50]:
model.wv.similarity(w1="cheap",w2="inexpensive")

0.5363185

In [52]:
model.wv.similarity(w1="great",w2="good")

0.7884099